# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene. You may define your own blender version.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.
* `optix_enable` : Enable OptiX which may boost performance, may be incompatible depending on the version of blender, project and GPU allocated
___
* `create_video` : Toggle whether to convert rendered frames into a video.
* `video_format` : Select the output format for the video (e.g., mp4, avi).
* `video_codec` : Choose the video codec for compression (e.g., libx264, libx265).
* `video_quality` : Set the quality preset for the video ('low', 'medium', 'high', 'lossless').
* `framerate` : Specify the framerate (frames per second) for the output video.
* `start_frame_video` : Define the starting frame number for the video sequence.
* `output_mode` : Choose whether to `keep_both` frames and video, `video_only`, or `frames_only` after conversion.
* `video_output_name` : Name of the output video file (e.g., `blender_render.mp4`).

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more information.

In [16]:
blender_version = '4.5.3' #@param ['2.79b', '2.83.20', '2.93.18', '3.3.21', '3.6.23', '4.2.14', '4.5.3'] {allow-input: true}
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'url' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'path/to/blend.zip' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = True #@param {type: 'boolean'}
start_frame =  0#@param {type: 'integer'}
end_frame =  360#@param {type: 'integer'}
#@markdown ---
download_type = 'gdrive_direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'output' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
#@markdown ---
create_video = True #@param {type: "boolean"}
video_format = "mp4" #@param ["mp4", "avi", "mov", "mkv", "webm"]
video_codec = "libx264" #@param ["libx264", "libx265", "mpeg4", "libvpx-vp9"]
video_quality = "high" #@param ["low", "medium", "high", "lossless"]
framerate = 24 #@param {type: "integer"}
start_frame_video = 0 #@param {type: "integer"}
#@markdown ---
output_mode = "keep_both" #@param ["keep_both", "video_only", "frames_only"]
video_output_name = "blender_render.mp4" #@param {type: "string"}

In [ ]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

In [ ]:
import shutil
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/MyDrive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

In [ ]:
!rm -r render
!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/MyDrive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")

In [ ]:
import requests
blender_url_dict = {
    '2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
    '2.83.20' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz"
}

if blender_version in blender_url_dict:
    blender_url = blender_url_dict[blender_version]
else:
    major_minor = ".".join(blender_version.split('.')[:2])
    blender_url = f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major_minor}/blender-{blender_version}-linux-x64.tar.xz"

try:
    response = requests.head(blender_url, allow_redirects=True, timeout=10)
    if response.status_code != 200:
        print(f"Download failed for version '{blender_version}'.")
        print("Error downloading: You may need to define the download archive manually above.")
    else:
        base_url = os.path.basename(blender_url)
        print(f"Download URL: {blender_url}")
        print(f"Base filename: {base_url}")
except Exception as e:
    print(f"Error checking URL: {e}")
    print("Error downloading: You may need to define the download archive manually above.")

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

In [ ]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (e.g. Tesla K80) is not supported and you need to switch back to CUDA.")
    renderer = "OPTIX"

In [ ]:
%cd /content

!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = '/content/output/' + output_name
else:
    output_path = '/drive/MyDrive/' + drive_output_path + output_name

%cd /content/$blender_version

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"

In [ ]:
%cd /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/MyDrive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/MyDrive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
else:
    for f in files_folder:
      shutil.copy("/content/output/" + f, '/drive/MyDrive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

# Video Conversion
Convert rendered frames into a video file.

(If you have already rendered your frames and just want to convert them to a video, Google Drive is mounted here too.)

In [ ]:
import subprocess
import glob
import os
from google.colab import files, drive
import re

drive.mount("/drive")

%cd /content

if create_video:
    print("Installing FFmpeg...")
    !apt-get update -qq
    !apt-get install -y ffmpeg -qq

    if download_type != 'gdrive_direct':
        frames_location = '/content/output/'
    else:
        frames_location = '/drive/MyDrive/' + drive_output_path
        if not frames_location.endswith('/'):
            frames_location += '/'

    base_name = output_name.replace('#', '')
    frame_name_pattern = base_name + '*'

    frame_files = sorted(glob.glob(os.path.join(frames_location, frame_name_pattern)))

    if len(frame_files) > 1:
        print(f"Found {len(frame_files)} frames to convert...")

        first_frame_path = frame_files[0]
        first_frame_name = os.path.basename(first_frame_path)
        frame_ext = os.path.splitext(first_frame_name)[1]

        numeric_part = ''
        if first_frame_name.startswith(base_name) and first_frame_name.endswith(frame_ext):
            numeric_part = first_frame_name[len(base_name):-len(frame_ext)]

        input_method_used = "concat"

        if numeric_part.isdigit():
            padding = len(numeric_part)
            ffmpeg_start_num = int(numeric_part)

            input_pattern = os.path.join(frames_location, f"{base_name}%0{padding}d{frame_ext}")

            ffmpeg_cmd_base = [
                'ffmpeg', '-y',
                '-r', str(framerate),
                '-start_number', str(ffmpeg_start_num),
                '-i', input_pattern,
                '-c:v', video_codec
            ]
            input_method_used = "image_sequence"
            print(f"Using image sequence input: {input_pattern}, starting from frame {ffmpeg_start_num}")
        else:
            print("Warning: Could not determine frame padding from numeric part. Falling back to concat list. This might be less robust for image sequences.")
            temp_list = '/tmp/frames_list.txt'
            with open(temp_list, 'w') as f:
                for frame_file in frame_files:
                    f.write(f"file '{frame_file}'\n")

            ffmpeg_cmd_base = [
                'ffmpeg', '-y',
                '-f', 'concat', '-safe', '0', '-i', temp_list,
                '-framerate', str(framerate),
                '-c:v', video_codec
            ]
            print(f"Using concat list input: {temp_list}")


        quality_presets = {
            'libx264': {'low': ['-crf', '28'], 'medium': ['-crf', '23'], 'high': ['-crf', '18'], 'lossless': ['-crf', '0']},
            'libx265': {'low': ['-crf', '32'], 'medium': ['-crf', '28'], 'high': ['-crf', '24'], 'lossless': ['-crf', '0']},
            'mpeg4': {'low': ['-q:v', '10'], 'medium': ['-q:v', '5'], 'high': ['-q:v', '2'], 'lossless': ['-q:v', '1']},
            'libvpx-vp9': {'low': ['-crf', '40', '-b:v', '0'], 'medium': ['-crf', '31', '-b:v', '0'], 'high': ['-crf', '23', '-b:v', '0'], 'lossless': ['-lossless', '1']}
        }

        quality_settings = quality_presets.get(video_codec, {}).get(video_quality, ['-crf', '23'])

        video_output = os.path.join(frames_location, video_output_name)

        ffmpeg_cmd = ffmpeg_cmd_base + quality_settings + ['-pix_fmt', 'yuv420p', video_output]

        print(f"Creating video: {video_output_name}")
        print(f"Location: {frames_location}")
        print(f"Codec: {video_codec} | Quality: {video_quality} | Framerate: {framerate} fps")
        print(f"Full FFmpeg command: {' '.join(ffmpeg_cmd)}")


        try:
            subprocess.run(ffmpeg_cmd, check=True, capture_output=True, text=True)
            print(f"✓ Video created successfully: {video_output_name}")

            if output_mode == "video_only":
                print("Deleting original frames...")
                for frame_file in frame_files:
                    os.remove(frame_file)
                print("✓ Original frames deleted")

            if frames_location.startswith('/content/') and download_type != 'gdrive_direct':
                print("Downloading video...")
                files.download(video_output)

        except subprocess.CalledProcessError as e:
            print(f"✗ Error creating video: {e}")
            print(f"stderr: {e.stderr}")

    else:
        print(f"Not enough frames found in {frames_location}")
        print(f"Looking for pattern: {frame_name_pattern}")
        print(f"Files found: {len(frame_files)}")

else:
    print("Video conversion skipped")

## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020-2022 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```